In [2]:
import pandas as pd
import requests
from pprint import pprint
import json


In [3]:
WEATHER_URL = "https://api.open-meteo.com/v1/forecast"
AIR_URL = "https://air-quality-api.open-meteo.com/v1/air-quality"

cities = [
    ("Lisbon", 38.7167, -9.1333),
    ("Thessaloniki", 40.6436, 22.9309),
    ("Brussels", 50.8505, 4.3488),
    ("Barcelona", 41.3888, 2.1590),
    ("Berlin", 52.5244, 13.4105),
]

rows = []

for name, lat, lon in cities:
    # Weather
    w_params = {
        "latitude": lat,
        "longitude": lon,
        "current": "temperature_2m,relative_humidity_2m,weather_code,wind_speed_10m",
        "timezone": "Europe/Athens",
    }
    w = requests.get(WEATHER_URL, params=w_params, timeout=30)
    w.raise_for_status()
    cw = w.json().get("current", {})

    # Air quality
    a_params = {
        "latitude": lat,
        "longitude": lon,
        "current": "european_aqi,pm2_5,nitrogen_dioxide,ozone",
        "timezone": "Europe/Athens",
    }
    a = requests.get(AIR_URL, params=a_params, timeout=30)
    a.raise_for_status()
    ca = a.json().get("current", {})

    rows.append({
        "city": name,
        "lat": lat,
        "lon": lon,
        "time": cw.get("time"),

        # weather
        "temp_c": cw.get("temperature_2m"),
        "humidity_pct": cw.get("relative_humidity_2m"),
        "wind_speed_ms": cw.get("wind_speed_10m"),
        "weather_code": cw.get("weather_code"),

        # air quality
        "aqi_eu": ca.get("european_aqi"),
        "pm25": ca.get("pm2_5"),
        "no2": ca.get("nitrogen_dioxide"),
        "o3": ca.get("ozone"),
    })

df = pd.DataFrame(rows)
display(df)


,city,lat,lon,time,temp_c,humidity_pct,wind_speed_ms,weather_code,aqi_eu,pm25,no2,o3
0,Lisbon,38.7167,-9.1333,2026-01-10T15:00,13.5,64,2.2,3,21,11.1,25.7,47.0
1,Thessaloniki,40.6436,22.9309,2026-01-10T15:00,9.5,85,7.4,3,26,5.3,8.8,60.0
2,Brussels,50.8505,4.3488,2026-01-10T15:00,-0.3,70,11.2,3,17,9.3,20.0,43.0
3,Barcelona,41.3888,2.1590,2026-01-10T15:00,12.6,46,15.5,1,27,2.5,7.0,67.0
4,Berlin,52.5244,13.4105,2026-01-10T15:00,-2.5,72,6.8,0,24,6.9,15.0,61.0


In [ ]:
df.to_csv("../data/raw/raw_weather_air.csv", index=False)